In [ ]:
# This file reads the raw file in and filter all non-english videos and videos published in countries other than "US", "CA", "GB", "AU", "unknown"

In [12]:
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
import json
import pandas as pd
import re
from bs4 import BeautifulSoup
from langdetect import detect
from pprint import pprint
import math
import os 
from scipy.stats import sem

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Phrases
from gensim import corpora, models
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import datapath
import pyLDAvis.gensim

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

import numpy as np

nltk.download('wordnet')
stemmer = SnowballStemmer("english")
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to C:\Users\Cat
[nltk_data]     Mai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
# gen tokens for each video. merge title and tags
# Filter the data and generate token lists

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def isEnglish(s):
    s=deEmojify(s)
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
def process_text(txt):
    result=str(txt)
    soup = BeautifulSoup(result)
    result=soup.get_text()
    result=re.sub('[,\.!?]', '', str(result))
    result=result.lower()
    return result

def process_tag(tags):
    result = tags.strip('][').split(',') 
    result=[r.strip('\' ') for r in result]
    result=" ".join(result)
    return result

def lemmatize_stemming(tokens):
    tks=[t.lower() for t in tokens if len(t)>=3]
    result=filter(lambda t: t not in STOPWORDS, tks)
    result=[lemmatizer.lemmatize(tk, pos='v') for tk in result]
#     result=[stemmer.stem(tk) for tk in result]
    return result


df=pd.read_csv("YouTube_CSV/video_FINAL_merged_teamtrees.csv")
print(len(df))    
sdf=df.loc[df.country.isin(["US", "CA", "GB", "AU", "NZ", "unknown"])]# only in English speaking countries
# sdf=df.loc[df.country.isin(["US", "CA", "GB", "AU", "unknown"])].iloc[:5000,:]# only in English speaking countries
texts=sdf.title.apply(process_text)
tags=sdf.tags.apply(process_tag)
corpus=[(text+" "+tag) for text,tag in zip(texts, tags)]
corpus_df=pd.DataFrame(data={"videoId":sdf["videoId"],"text":corpus})
corpus_df=corpus_df.loc[corpus_df.text.apply(isEnglish)] # must by English
corpus_df=corpus_df.loc[corpus_df.text.apply(lambda x: "teamtrees" in x.lower())] # must have asmr

corpus_df["tokens"]=corpus_df.text.apply(word_tokenize)
corpus_df["tokens"]=corpus_df.tokens.apply(lemmatize_stemming) # lower the token and lemmatize
bigram = Phrases(corpus_df.tokens, min_count=len(corpus_df)*0.05, delimiter=b'_')
corpus_df["tokens"]=corpus_df.tokens.apply(lambda tk: bigram[tk])

# Save the dictionary
dictionary = gensim.corpora.Dictionary(corpus_df["tokens"])
dictionary.filter_extremes(no_below=20, no_above=0.5)
dictionary.save("dictionary")
bow=list(corpus_df.tokens.apply(lambda tks:dictionary.doc2bow(tks)))

corpus_df.to_csv("YouTube_CSV/corpus_data.csv", index=False)

print(len(corpus_df))
corpus_df

<>:26: DeprecationWarning: invalid escape sequence \.
<>:26: DeprecationWarning: invalid escape sequence \.
<>:26: DeprecationWarning: invalid escape sequence \.
<ipython-input-13-495bccfd7225>:26: DeprecationWarning: invalid escape sequence \.
  result=re.sub('[,\.!?]', '', str(result))


1445
937


,videoId,text,tokens
1,:-5VBe9i4HDg,live tree planting counter - teamtrees [20 mil...,"[live, tree, plant, counter, teamtrees, millio..."
3,:-99jZt79aWk,why are trees so awesome #teamtrees nanodots m...,"[tree, awesome, teamtrees, nanodots, magnets, ..."
4,:-BqH2PKqFpM,the tree face #teamtrees robisplays teamtrees ...,"[tree, face, teamtrees, robisplays, teamtrees,..."
5,:-BRWSaYQhM0,donate to teamtrees #teamtrees,"[donate, teamtrees, teamtrees]"
7,:-cPdImejxEQ,planting 20000000 trees will actually have thi...,"[plant, 20000000, tree, actually, impact, mrbe..."
...,...,...,...
1437,:_rQrNfRww60,hallowtree | donate to team trees | Animation ...,"[hallowtree, donate, team, tree, animation, ha..."
1439,:_vQ7OeVWBrU,let's stream for teamtrees gaming pixelfire ai...,"[let, stream, teamtrees, game, pixelfire, code..."
1442,:_yKFJm75qO0,planting 20 million trees in prison architect ...,"[plant, million, tree, prison, architect, simu..."
1443,:_zT8aOvNLaY,asmr | bob ross inspired painting asmr for #te...,"[asmr, bob, ross, inspire, paint, asmr, teamtr..."


In [22]:
n = 50
ROUND = 3
import pandas as pd
import random
import os


#read tagged urls
df_tagged = pd.read_csv("Snow_Tagging/snowtagged.csv")
assert(df_tagged['videoId'].is_unique)
print(len(df_tagged))
tagged_id_list = [i for i in df_tagged["videoId"]]

videoid = list(corpus_df["videoId"])
videoid = [i for i in videoid if i is not "#NAME?" and i not in tagged_id_list]
random.shuffle(videoid)


#read from final data and shuffle video id list, removing invalid id and tagged id
df = pd.read_csv("YouTube_CSV/video_FINAL_merged_teamtrees.csv")

cols = ["embedUrl",
       "videoId",
       "title",
       "isIncluded",
       "diagnostic",
       "prognostic",
       "motivational"]

#convert to df and save to csv, add them to snowtagged.csv
cat_ids = [i for i in videoid[:n]]
# niu_ids = [i for i in videoid[n:n*2]]
# kathy_ids = [i for i in videoid[n*2:n*3]]

def generate_df(id_list):
    data = []
    embedUrl = ["https://www.youtube.com/embed/"+i[1:] for i in id_list]
    title = [str(df.loc[df["videoId"]==i]["title"].item()) for i in id_list]
    videoId = [i for i in id_list]
    data = [embedUrl, videoId, title, [], [], [], []]
    return data

df_cat = pd.DataFrame(generate_df(cat_ids)).transpose()
# df_niu = pd.DataFrame([["https://www.youtube.com/watch?v="+i for i in niu_ids],[],[],[]]).transpose()
# df_kathy = pd.DataFrame([["https://www.youtube.com/watch?v="+i for i in kathy_ids],[],[],[]]).transpose()
df_cat.columns = cols
# df_niu.columns = cols
# df_kathy.columns = cols
df_cat.to_csv(f"Snow_Tagging/SnowTagging_Cat_ROUND{ROUND}.csv", index=False)
df_cat.to_csv(f"Snow_Tagging/SnowTagging_Niu_ROUND{ROUND}.csv", index=False)
df_cat.to_csv(f"Snow_Tagging/SnowTagging_Kathy_ROUND{ROUND}.csv", index=False)
# df_niu.to_csv(f"SnowTagging_Niu_ROUND{ROUND}.csv", index=False)
# df_kathy.to_csv(f"SnowTagging_Kathy_ROUND{ROUND}.csv", index=False)



120


In [24]:
new_urls = ["https://www.youtube.com/embed/"+i for i in cat_ids]
new_ids = [i for i in cat_ids]
df_new = pd.DataFrame([new_urls,new_ids]).transpose()
df_new.columns = ["video_url","videoId"]
df_new.to_csv("Snow_Tagging/snowtagged.csv", mode='a', header=False, index=False)